# Simple Offline DTW

This notebook implements a simple offline DTW baseline system.  The only requirement in this notebook is that it implement the `offline_processing()` and `online_processing()` functions, which will be imported and run in `02_RunExperiment.ipynb`.  The rest of the notebook is for experimenting, visualizing, and analyzing the system, so it should be thought of as a sandbox for development.

Here is a summary of the offline DTW approach:
- Offline processing: The orchestra and full mix recordings are aligned with standard DTW using chroma features.
- Online processing: The solo piano and full mix recordings are aligned with standard DTW using chroma features, and the predicted alignment is then used to infer the corresponding alignment between the piano and orchestra recordings.  Note that this approach is not actually a valid online approach -- it serves primarily as a reference to gauge the effectiveness of other approaches.

## Offline Processing

In the offline processing stage, three things are computed and stored in the `cache/` folder:
- chroma features for the orchestra recording
- chroma features for the full mix recording
- predicted DTW alignment between the orchestra and full mix recordings

In [ ]:
%matplotlib inline
%load_ext Cython

In [ ]:
import numpy as np
import librosa as lb
# import numpy as np
# import matplotlib.pyplot as plt
# import librosa as lb
import os.path
# import os
# import pickle

The following is a cython implementation of standard DTW.

In [ ]:
%%cython
import numpy as np
cimport numpy as np
cimport cython

import sys
import time


DTYPE_INT32 = np.int32
ctypedef np.int32_t DTYPE_INT32_t

DTYPE_FLOAT = np.float64
ctypedef np.float64_t DTYPE_FLOAT_t

cdef DTYPE_FLOAT_t MAX_FLOAT = float('inf')

# careful, without bounds checking can mess up memory - also can't use negative indices I think (like x[-1])
@cython.boundscheck(False) # turn off bounds-checking for entire function
def DTW_Cost_To_AccumCostAndSteps(Cin, parameter):
    '''
    Inputs
        C: The cost Matrix
    '''


    '''
    Section for checking and catching errors in the inputs
    '''

    cdef np.ndarray[DTYPE_FLOAT_t, ndim=2] C
    try:
        C = np.array(Cin, dtype=DTYPE_FLOAT)
    except TypeError:
        print(bcolors.FAIL + "FAILURE: The type of the cost matrix is wrong - please pass in a 2-d numpy array" + bcolors.ENDC)
        return [-1, -1, -1]
    except ValueError:
        print(bcolors.FAIL + "FAILURE: The type of the elements in the cost matrix is wrong - please have each element be a float (perhaps you passed in a matrix of ints?)" + bcolors.ENDC)
        return [-1, -1, -1]

    cdef np.ndarray[np.uint32_t, ndim=1] dn
    cdef np.ndarray[np.uint32_t, ndim=1] dm
    cdef np.ndarray[DTYPE_FLOAT_t, ndim=1] dw
    # make sure dn, dm, and dw are setup
    # dn loading and exception handling
    if ('dn'  in parameter.keys()):
        try:

            dn = np.array(parameter['dn'], dtype=np.uint32)
        except TypeError:
            print(bcolors.FAIL + "FAILURE: The type of dn (row steps) is wrong - please pass in a 1-d numpy array that holds uint32s" + bcolors.ENDC)
            return [-1, -1, -1]
        except ValueError:
            print(bcolors.FAIL + "The type of the elements in dn (row steps) is wrong - please have each element be a uint32 (perhaps you passed a long?). You can specify this when making a numpy array like: np.array([1,2,3],dtype=np.uint32)" + bcolors.ENDC)
            return [-1, -1, -1]
    else:
        dn = np.array([1, 1, 0], dtype=np.uint32)
    # dm loading and exception handling
    if 'dm'  in parameter.keys():
        try:
            dm = np.array(parameter['dm'], dtype=np.uint32)
        except TypeError:
            print(bcolors.FAIL + "FAILURE: The type of dm (col steps) is wrong - please pass in a 1-d numpy array that holds uint32s" + bcolors.ENDC)
            return [-1, -1, -1]
        except ValueError:
            print(bcolors.FAIL + "FAILURE: The type of the elements in dm (col steps) is wrong - please have each element be a uint32 (perhaps you passed a long?). You can specify this when making a numpy array like: np.array([1,2,3],dtype=np.uint32)" + bcolors.ENDC)
            return [-1, -1, -1]
    else:
        print(bcolors.FAIL + "dm (col steps) was not passed in (gave default value [1,0,1]) " + bcolors.ENDC)
        dm = np.array([1, 0, 1], dtype=np.uint32)
    # dw loading and exception handling
    if 'dw'  in parameter.keys():
        try:
            dw = np.array(parameter['dw'], dtype=DTYPE_FLOAT)
        except TypeError:
            print(bcolors.FAIL + "FAILURE: The type of dw (step weights) is wrong - please pass in a 1-d numpy array that holds floats" + bcolors.ENDC)
            return [-1, -1, -1]
        except ValueError:
            print(bcolors.FAIL + "FAILURE:The type of the elements in dw (step weights) is wrong - please have each element be a float (perhaps you passed ints or a long?). You can specify this when making a numpy array like: np.array([1,2,3],dtype=np.float64)" + bcolors.ENDC)
            return [-1, -1, -1]
    else:
        dw = np.array([1, 1, 1], dtype=DTYPE_FLOAT)
        print(bcolors.FAIL + "dw (step weights) was not passed in (gave default value [1,1,1]) " + bcolors.ENDC)

    
    '''
    Section where types are given to the variables we're going to use 
    '''
    # create matrices to store our results (D and E)
    cdef DTYPE_INT32_t numRows = C.shape[0] # only works with np arrays, use np.shape(x) will work on lists? want to force to use np though?
    cdef DTYPE_INT32_t numCols = C.shape[1]
    cdef DTYPE_INT32_t numDifSteps = np.size(dw)

    cdef unsigned int maxRowStep = max(dn)
    cdef unsigned int maxColStep = max(dm)

    cdef np.ndarray[np.uint32_t, ndim=2] steps = np.zeros((numRows,numCols), dtype=np.uint32)
    cdef np.ndarray[DTYPE_FLOAT_t, ndim=2] accumCost = np.ones((maxRowStep + numRows, maxColStep + numCols), dtype=DTYPE_FLOAT) * MAX_FLOAT

    cdef DTYPE_FLOAT_t bestCost
    cdef DTYPE_INT32_t bestCostIndex
    cdef DTYPE_FLOAT_t costForStep
    cdef unsigned int row, col
    cdef unsigned int stepIndex

    '''
    The start of the actual algorithm, now that all our variables are set up
    '''
    # initializing the cost matrix - depends on whether its subsequence DTW
    # essentially allow us to hop on the bottom anywhere (so could start partway through one of the signals)
    if parameter['SubSequence']:
        for col in range(numCols):
            accumCost[maxRowStep, col + maxColStep] = C[0, col]
    else:
        accumCost[maxRowStep, maxColStep] = C[0,0]

    # filling the accumulated cost matrix
    for row in range(maxRowStep, numRows + maxRowStep, 1):
        for col in range(maxColStep, numCols + maxColStep, 1):
            bestCost = accumCost[<unsigned int>row, <unsigned int>col] # initialize with what's there - so if is an entry point, then can start low
            bestCostIndex = 0
            # go through each step, find the best one
            for stepIndex in range(numDifSteps):
                #costForStep = accumCost[<unsigned int>(row - dn[<unsigned int>(stepIndex)]), <unsigned int>(col - dm[<unsigned int>(stepIndex)])] + dw[<unsigned int>(stepIndex)] * C[<unsigned int>(row - maxRowStep), <unsigned int>(col - maxColStep)]
                costForStep = accumCost[<unsigned int>((row - dn[(stepIndex)])), <unsigned int>((col - dm[(stepIndex)]))] + dw[stepIndex] * C[<unsigned int>(row - maxRowStep), <unsigned int>(col - maxColStep)]
                if costForStep < bestCost:
                    bestCost = costForStep
                    bestCostIndex = stepIndex
            # save the best cost and best cost index
            accumCost[row, col] = bestCost
            steps[<unsigned int>(row - maxRowStep), <unsigned int>(col - maxColStep)] = bestCostIndex

    # return the accumulated cost along with the matrix of steps taken to achieve that cost
    return [accumCost[maxRowStep:, maxColStep:], steps]

@cython.boundscheck(False) # turn off bounds-checking for entire function
def DTW_GetPath(np.ndarray[DTYPE_FLOAT_t, ndim=2] accumCost, np.ndarray[np.uint32_t, ndim=2] stepsForCost, parameter):
    '''

    Parameter should have: 'dn', 'dm', 'dw', 'SubSequence'
    '''

    cdef np.ndarray[unsigned int, ndim=1] dn
    cdef np.ndarray[unsigned int, ndim=1] dm
    cdef np.uint8_t subseq
    cdef np.int32_t startCol # added
    # make sure dn, dm, and dw are setup
    if ('dn'  in parameter.keys()):
        dn = parameter['dn']
    else:
        dn = np.array([1, 1, 0], dtype=DTYPE_INT32)
    if 'dm'  in parameter.keys():
        dm = parameter['dm']
    else:
        dm = np.array([1, 0, 1], dtype=DTYPE_INT32)
    if 'SubSequence' in parameter.keys():
        subseq = parameter['SubSequence']
    else:
        subseq = 0
    
    # added START
    if 'startCol' in parameter.keys(): 
        startCol = parameter['startCol']
    else:
        startCol = -1
    # added END

    cdef np.uint32_t numRows
    cdef np.uint32_t numCols
    cdef np.uint32_t curRow
    cdef np.uint32_t curCol
    cdef np.uint32_t endCol
    cdef DTYPE_FLOAT_t endCost

    numRows = accumCost.shape[0]
    numCols = accumCost.shape[1]

    # either start at the far corner (non sub-sequence)
    # or start at the lowest cost entry in the last row (sub-sequence)
    # where all of the signal along the row has been used, but only a 
    # sub-sequence of the signal along the columns has to be used
    curRow = numRows - 1
    if subseq:
        curCol = np.argmin(accumCost[numRows - 1, :])
    else:
        curCol = numCols - 1
        
    # added - if specified, overrides above
    if startCol >= 0:
        curCol = startCol

    endCol = curCol
    endCost = accumCost[curRow, curCol]

    cdef np.uint32_t curRowStep
    cdef np.uint32_t curColStep
    cdef np.uint32_t curStepIndex


    cdef np.ndarray[np.uint32_t, ndim=2] path = np.zeros((2, numRows + numCols), dtype=np.uint32) # make as large as could need, then chop at the end
    path[0, 0] = curRow
    path[1, 0] = curCol

    cdef np.uint32_t stepsInPath = 1 # starts at one, we add in one before looping
    cdef np.uint32_t stepIndex = 0
    cdef np.int8_t done = (subseq and curRow == 0) or (curRow == 0 and curCol == 0)
    while not done:
        if accumCost[curRow, curCol] == MAX_FLOAT:
            print('A path is not possible')
            break

        # you're done if you've made it to the bottom left (non sub-sequence)
        # or just the bottom (sub-sequence)
        # find the step size
        curStepIndex = stepsForCost[curRow, curCol]
        curRowStep = dn[curStepIndex]
        curColStep = dm[curStepIndex]
        # backtrack by 1 step
        curRow = curRow - curRowStep
        curCol = curCol - curColStep
        # add your new location onto the path
        path[0, stepsInPath] = curRow
        path[1, stepsInPath] = curCol
        stepsInPath = stepsInPath + 1
        # check to see if you're done
        done = (subseq and curRow == 0) or (curRow == 0 and curCol == 0)

    # reverse the path (a matrix with two rows) and return it
    return [np.fliplr(path[:, 0:stepsInPath]), endCol, endCost]

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [ ]:
def verify_scenario_dir(indir):
    '''
    Verifies that the specified scenario directory has the required files.
    
    Inputs
    indir: the scenario directory to verify
    '''
    assert os.path.exists(indir)
    assert os.path.exists(f'{indir}/p.wav')
    assert os.path.exists(f'{indir}/o.mp3')
    assert os.path.exists(f'{indir}/po.mp3')

In [ ]:
def compute_dtw_alignment(F1, F2, steps, weights, subseq = False):
    '''
    Wrapper function for computing a DTW alignment.
    
    Inputs
    F1: Feature matrix for first recording, specified as a 12 x N matrix
    F2: Feature matrix for second recording, specified as a 12 x M matrix
    steps: L x 2 numpy array specifying the allowable DTW transitions
    weights: length L array specifying the DTW transition weights
    subseq: boolean value indicating whether or not to perform subsequence DTW
    
    Returns the estimated alignment path, specified as a K x 2 numpy array.
    '''
    C = 1 - F1.T @ F2 # cos distance metric
    dn = steps[:,0].astype(np.uint32)
    dm = steps[:,1].astype(np.uint32)
    parameters = {'dn': dn, 'dm': dm, 'dw': weights, 'SubSequence': subseq}
    [D, s] = DTW_Cost_To_AccumCostAndSteps(C, parameters)
    [wp, endCol, endCost] = DTW_GetPath(D, s, parameters)

    return wp

In [ ]:
def offline_processing(scenario_dir, cache_dir, hop_length, steps, weights):
    '''
    Carries out offline processing for a simple offline DTW system.
    
    Inputs
    scenario_dir: The scenario directory to process
    cache_dir: The location of the cache directory
    hop_length: The hop length in samples used when computing chroma features
    steps: an L x 2 array specifying the allowable DTW transitions
    weights: a length L array specifying the DTW transition weights
    
    This function will store the computed chroma features and estimated alignment in the cache folder.
    '''
    
    # setup
    verify_scenario_dir(scenario_dir)
    if os.path.exists(cache_dir):
        print(f'{cache_dir} has already been processed.  Skipping.')
        pass
    else:
        # setup
        os.makedirs(cache_dir)

        # compute orchestra features
        o_file = f'{scenario_dir}/o.mp3'
        y_o, sr = lb.core.load(o_file)
        F_o = lb.feature.chroma_cqt(y_o, sr=sr, hop_length=hop_length, norm=2) 

        # compute full mix features
        po_file = f'{scenario_dir}/po.mp3'
        y_po, sr = lb.core.load(po_file)
        F_po = lb.feature.chroma_cqt(y_po, sr=sr, hop_length=hop_length, norm=2)
      
        # compute alignment
        wp = compute_dtw_alignment(F_po, F_o, steps, weights)

        # save to cache
        np.save(f'{cache_dir}/o_chroma.npy', F_o)
        np.save(f'{cache_dir}/po_chroma.npy', F_po)
        np.save(f'{cache_dir}/po_o_align.npy', wp)
    
    return

# Online Processing

In the online processing stage, we do two things:
- compute an offline DTW alignment between the piano and full mix recordings,
- use the predicted alignment to infer the alignment between the piano and orchestra recordings

Note that this baseline system is not a valid online system since it uses offline DTW.  It merely serves as a reference comparison.

In [ ]:
def verify_cache_dir(indir):
    '''
    Verifies that the specified cache directory has the required files.
    
    Inputs
    indir: The cache directory to verify
    '''
    assert os.path.exists(f'{indir}/o_chroma.npy')
    assert os.path.exists(f'{indir}/po_chroma.npy')
    assert os.path.exists(f'{indir}/po_o_align.npy')

In [ ]:
def infer_alignment(wp1, wp2):
    '''
    Given alignments between recording pairs (A, B) and (B, C), infer the estimated alignment
    between the pair (A, C).
    
    Inputs
    wp1: The first warping path, specified as a 2 x N numpy array
    wp2: The second warping path, specified as a 2 x M numpy array
    
    Returns a 2 x L array containing the inferred alignment.
    '''
    
    frames_A = np.arange(wp1[0,-1] + 1) # all A frames
    frames_B_interp = np.interp(frames_A, wp1[0,:], wp1[1,:]) # infer corresponding B frames
    frames_C_interp = np.interp(frames_B_interp, wp2[0,:], wp2[1,:]) # infer corresponding C frames
    
    return np.vstack((frames_A, frames_C_interp))

In [ ]:
def online_processing(scenario_dir, out_dir, cache_dir, hop_length, steps, weights):
    '''
    Carries out `online' processing for a simple offline DTW system.
    
    Inputs
    scenario_dir: The scenario directory to process
    out_dir: The directory to put results, intermediate files, and logging info
    cache_dir: The cache directory
    hop_length: The hop length in samples used when computing chroma features
    steps: an L x 2 array specifying the allowable DTW transitions
    weights: a length L array specifying the DTW transition weights

    This function will compute and save the predicted alignment in the output directory in a file hyp.npy
    '''
    
    # verify & setup
    verify_scenario_dir(scenario_dir)
    verify_cache_dir(cache_dir)
    assert not os.path.exists(out_dir)
    os.makedirs(out_dir)
    
    # compute features
    p_file = f'{scenario_dir}/p.wav'
    y, sr = lb.core.load(p_file)
    F_p = lb.feature.chroma_cqt(y, sr=sr, hop_length=hop_length, norm=2)  # piano features
    F_po = np.load(f'{cache_dir}/po_chroma.npy') # full mix features
    
    # compute alignments
    wp_AB = compute_dtw_alignment(F_p, F_po, steps, weights, subseq=True) 
    wp_BC = np.load(f'{cache_dir}/po_o_align.npy')
    wp_AC = infer_alignment(wp_AB, wp_BC) # piano-orchestra alignment
    
    # save results
    np.save(f'{out_dir}/hyp.npy', wp_AC)
    
    return

# Example

Here is an example of how to call the offline and online processing functions on a scenario directory.

In [ ]:
# scenario_dir = 'scenarios/s2'
# out_dir = 'experiments/offlineDTW/s2'
# cache_dir = 'experiments/offlineDTW/cache2'
# hop_size = 512
# steps = np.array([1,1,1,2,2,1]).reshape((-1,2))
# weights = np.array([2,3,3])
# offline_processing(scenario_dir, cache_dir, hop_size, steps, weights)
# online_processing(scenario_dir, out_dir, cache_dir, hop_size, steps, weights)